In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis2 as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

/Users/edasmalchi/miniconda3/envs/calitp/lib/python3.10/site-packages/google/cloud/bigquery/client.py:535: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [2]:
# analysis_date = dt.date(2022, 1, 11)
## all data incomplete -- did it crash? (nothing before 9ish)

In [3]:
analysis_date = dt.date(2022, 1, 12)

In [4]:
samtrans_itp_id = 290

In [5]:
# utils.get_vehicle_positions(samtrans_itp_id, analysis_date)

In [6]:
# utils.get_trips(samtrans_itp_id, analysis_date)

In [7]:
# utils.get_stop_times(samtrans_itp_id, analysis_date, force_clear=False)

In [8]:
# utils.get_stops(samtrans_itp_id, analysis_date)

In [9]:
# sam_trips >> head(5)

In [10]:
# sam_vp >> head(5)

In [11]:
# sam_vp >> inner_join(_, sam_trips, on ='trip_id')

In [12]:
## TODO
# lbt_analysis.position_interpolators['9288623']['rt'].trip_key
# generated 1553149201202261125 on 11/30/21 but this doesn't seem to exist?

In [5]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [7]:
air_joined['url'] = air_joined.apply(lambda x:
                    f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_01_12_pm_peak.html',
                                    axis = 1)

In [8]:
air_joined.to_csv('linked.csv')

In [14]:
# for agency in air_joined.calitp_itp_id:
#     try:
#         analysis = rt.OperatorDayAnalysis(agency, analysis_date)
#     except Exception as e:
#         print(f'failed on agency {agency}')
#         print(e)

In [15]:
## fix routelines thing...

In [16]:
# metro_analysis.position_interpolators['10105002581045-DEC21']['rt'].detailed_speed_map()

In [17]:
# lbt_analysis.position_interpolators['9288672']['rt'].detailed_speed_map()

In [18]:
# lbt = rt.OperatorDayAnalysis(170, analysis_date)

In [21]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis2' from '/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py'>

In [22]:
# rt.OperatorDayAnalysis.map_segment_speeds(lbt, how = 'average')

In [23]:
# lbt.segment_speed_view['direction'] = '*'

In [24]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [25]:
# lbt._generate_segment_map(how = 'average', colorscale=colorscale)

In [26]:
## TODO '11031623-136-Blocks-Weekday-54' should not be eastbound... (Metro?)

In [3]:
# sam = rt.OperatorDayAnalysis(samtrans_itp_id, analysis_date)

In [29]:
(sam.stop_delay_view
 >> group_by(_.stop_id, _.stop_name)
 >> summarize(mean_delay = _.delay.mean(), max_delay = _.delay.max())
 >> arrange(-_.mean_delay))

,stop_id,stop_name,mean_delay,max_delay
711,334620,South SF BART-Bay 5 Westside Busway,0 days 00:26:28.206070,0 days 00:26:28.206070
1188,343118,Alameda de las Pulgas & Alma St,0 days 00:18:39.587416400,0 days 00:22:35.861996
1350,344637,Redwood City Caltrain-Lane C,0 days 00:18:09.367815400,0 days 01:11:28.602062
265,331100,SF Transit Center - BART shuttle connection,0 days 00:17:02.685705,0 days 00:17:02.685705
41,311106,Linda Mar Park N Ride,0 days 00:17:00.310031461,0 days 00:35:10.992110
...,...,...,...,...
1695,363051,Pulgas Ave & O'Connor St,0 days 00:00:00,0 days 00:00:00
1697,363053,Pulgas Ave & Sage St,0 days 00:00:00,0 days 00:00:00
1709,363101,East Okeefe St & Menalto Ave,0 days 00:00:00,0 days 00:00:00
1711,363103,East Okeefe St & Euclid Ave,0 days 00:00:00,0 days 00:00:00


In [30]:
## try filter!

In [50]:
sam.set_filter(start_time='15:00', end_time='19:00', direction_id='1')

In [51]:
sam.filter

{'start_time': datetime.time(15, 0),
 'end_time': datetime.time(19, 0),
 'route_ids': None,
 'direction_id': '1',
 'direction': None}

In [52]:
%%capture
m = sam.segment_speed_map(colorscale=colorscale)

In [56]:
# m

In [2]:
# lbt = rt.OperatorDayAnalysis(170, analysis_date)

In [ ]:
lbt.set_filter(direction_id='0')

In [60]:
%%capture
lbt._generate_stop_delay_view()

In [74]:
lbt.set_filter(start_time='15:00', end_time='19:00', route_ids=['175'], direction_id='0')

In [75]:
lbt.filter

{'start_time': datetime.time(15, 0),
 'end_time': datetime.time(19, 0),
 'route_ids': ['175'],
 'direction_id': '0',
 'direction': None}

In [79]:
lbt._filter(lbt.rt_trips)

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': ['175'], 'direction_id': '0', 'direction': None}


,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,median_time,direction
0,170,0,2022-01-12,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,17:25:56,Westbound
1,170,0,2022-01-12,6004281467132627200,9288619,175,0,1750012,2021-09-16,2099-01-01,16:08:22,Westbound
2,170,0,2022-01-12,-6507058681082383946,9288621,175,0,1750012,2021-09-16,2099-01-01,16:47:36,Westbound
3,170,0,2022-01-12,-3729871019479677706,9288617,175,0,1750012,2021-09-16,2099-01-01,15:26:50,Westbound
4,170,0,2022-01-12,6062701308686180855,9288625,175,0,1750012,2021-09-16,2099-01-01,18:08:08,Westbound
5,170,0,2022-01-12,6978398524389426859,9288627,175,0,1750012,2021-09-16,2099-01-01,18:49:15,Westbound


In [1]:
# %%capture
# m = lbt.segment_speed_map(how='low_speeds', colorscale=colorscale)